<a href="https://colab.research.google.com/github/randaama/DS4002_Group2_Project2/blob/main/SCRIPTS/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sooo we need to look at the trends of the top  10 Counties
Orange County  
Miami Dade County
Hennepin County
Hillsborough County
Broward County
Maricopa County
Hamilton County
Pinellas County
Natrona County
Fullton county
to give a desirability determination based on precipitation and average temperature. Use that to build a desiribity() function 1. Predict the average temperature and precipitation of each month based on the change in the past and then stick that into the desirability function and then rank the top Counties.

In [207]:
import pandas as pd
import plotly.express as px
#import pyearth
clim = pd.read_csv('/Users/selahmitchell/climate_data_all.csv')

In [208]:
import plotly.express as px
import numpy as np

In [94]:
clim.head()

,year,fips,month,temp,County_Name,State,percip
0,2004,1001,Jan,45.0,Autauga,Alabama,2.60
1,2005,1001,Jan,49.9,Autauga,Alabama,3.08
2,2006,1001,Jan,52.0,Autauga,Alabama,5.69
3,2007,1001,Jan,47.3,Autauga,Alabama,4.73
4,2008,1001,Jan,43.4,Autauga,Alabama,4.36


In [217]:
new_counties_data = [
    {"Rank": 1, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 2, "County_Name": "Hennepin", "State": "Minnesota"},
    {"Rank": 3, "County_Name": "Hillsborough", "State": "Florida"},
    {"Rank": 4, "County_Name": "Broward", "State": "Florida"},
    {"Rank": 5, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 6, "County_Name": "Hamilton", "State": "Ohio"},
    {"Rank": 7, "County_Name": "Pinellas", "State": "Florida"},
    {"Rank": 8, "County_Name": "Natrona", "State": "Wyoming"},
    {"Rank": 9, "County_Name": "Fulton", "State": "Georgia"},
]
# Create DataFrame for new counties
new_counties_df = pd.DataFrame(new_counties_data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_wallet = pd.merge(clim, new_counties_df, on=['County_Name', 'State'], how='inner')

# Print the resulting DataFrame
print(top10_wallet)

      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     5
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     5
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     5
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     5
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     5
...    ...    ...   ...   ...         ...      ...     ...   ...
2155  2019  56025   Dec  25.0     Natrona  Wyoming    0.51     8
2156  2020  56025   Dec  25.5     Natrona  Wyoming    0.62     8
2157  2021  56025   Dec  27.7     Natrona  Wyoming    0.41     8
2158  2022  56025   Dec  21.0     Natrona  Wyoming    0.66     8
2159  2023  56025   Dec  30.3     Natrona  Wyoming    0.14     8

[2160 rows x 8 columns]


In [220]:
import pandas as pd

top10_wallet['year_month'] = top10_wallet['year'].astype(str) + '-' + top10_wallet['month']

# Calculate overall averages for each month across all years
overall_avg = top10_wallet.groupby(['month', 'year']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
overall_avg.columns = ['month', 'year', 'overall_avg_temp', 'overall_avg_percip']

# Merge overall averages with the monthly data
monthly_averages = pd.merge(top10_wallet, overall_avg, on=['month', 'year'])

    # Calculate absolute differences from overall averages
monthly_averages['temp_diff'] = abs(monthly_averages['temp'] - monthly_averages['overall_avg_temp'])
monthly_averages['percip_diff'] = abs(monthly_averages['percip'] - monthly_averages['overall_avg_percip'])

# Weights for desirability components
w_T = 0.3  # weight for temperature
w_P = 0.2  # weight for precipitation
w_r = 0.5  # weight for rank

# Normalize the rank so that a lower rank yields a higher desirability contribution
monthly_averages['rank_adjusted'] = monthly_averages['Rank'].max() - monthly_averages['Rank'] + 1

# Calculate the desirability score for each month
monthly_averages['desirability_month'] = (
    w_T * monthly_averages['temp_diff'] +
    w_P * monthly_averages['percip_diff'] +
    w_r * monthly_averages['rank_adjusted']
    )

# Calculate yearly desirability score by taking the mean of monthly scores per county and year
yearly_desirability = monthly_averages.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']

# Merge yearly desirability back into the monthly averages DataFrame
monthly_averages = pd.merge(monthly_averages, yearly_desirability, on=['County_Name', 'State', 'year'], how='left')

# Output the final results
final_results = monthly_averages[['County_Name', 'State', 'temp', 'percip', 'desirability_month', 'Rank', 'month', 'year', 'desirability_year']]
print(final_results)

       County_Name      State  temp  percip  desirability_month  Rank month  \
0         Maricopa    Arizona  53.9    0.52            5.520000     5   Jan   
1          Broward    Florida  64.9    2.10            9.004000     4   Jan   
2     Hillsborough    Florida  59.0    3.64            7.900667     3   Jan   
3       Miami-Dade    Florida  65.7    2.28           10.708000     1   Jan   
4         Pinellas    Florida  59.1    4.49            6.100667     7   Jan   
...            ...        ...   ...     ...                 ...   ...   ...   
2155      Pinellas    Florida  63.7    3.62            5.018000     7   Dec   
2156        Fulton    Georgia  47.2    3.95            2.347333     9   Dec   
2157      Hennepin  Minnesota  32.1    2.31           10.224000     2   Dec   
2158      Hamilton       Ohio  41.2    2.09            5.538000     6   Dec   
2159       Natrona    Wyoming  30.3    0.14            8.198000     8   Dec   

      year  desirability_year  
0     2004         

In [98]:

# Maybe contrast this wallethub (financially best ) with some other list of where people actually moved
#and look at the temperature differences between the 2 as welll as the "desired"
# Weight the scores based off of temp and precip 60/40
top10[['month','temp']].groupby('month').mean()

,temp
month,
Apr,64.70
Aug,81.02
Dec,54.72
Feb,55.02
Jan,51.33
Jul,81.42
Jun,75.69
Mar,57.45
May,70.85


In [99]:
top10[['month','percip']].groupby('month').mean()

,percip
month,
Apr,3.585
Aug,4.637
Dec,3.174
Feb,2.265
Jan,2.524
Jul,4.123
Jun,5.469
Mar,2.797
May,3.290


In [113]:
data = [
    {"Rank": 1, "County_Name": "Maricopa", "State": "Arizona"},
    {"Rank": 2, "County_Name": "Clark", "State": "Nevada"},
    {"Rank": 3, "County_Name": "Bexar", "State": "Texas"},
    {"Rank": 4, "County_Name": "Ada", "State": "Idaho"},
    {"Rank": 6, "County_Name": "New Hanover", "State": "North Carolina"},
    {"Rank": 7, "County_Name": "Collin", "State": "Texas"},
    {"Rank": 8, "County_Name": "Miami-Dade", "State": "Florida"},
    {"Rank": 9, "County_Name": "Duval", "State": "Florida"},
    {"Rank": 10, "County_Name": "Wake", "State": "North Carolina"}
]
counties_df = pd.DataFrame(data)

# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)


In [114]:
# Merge climate data with counties based on 'County_Name' and 'State'
top10_migration= pd.merge(clim, counties_df, on=['County_Name', 'State'], how='inner')
print(top10_migration)

      year   fips month  temp County_Name    State  percip  Rank
0     2004   4013   Jan  53.9    Maricopa  Arizona    0.52     1
1     2005   4013   Jan  54.3    Maricopa  Arizona    2.45     1
2     2006   4013   Jan  54.0    Maricopa  Arizona    0.00     1
3     2007   4013   Jan  49.8    Maricopa  Arizona    0.68     1
4     2008   4013   Jan  51.1    Maricopa  Arizona    2.19     1
...    ...    ...   ...   ...         ...      ...     ...   ...
2155  2019  48085   Dec  48.3      Collin    Texas    1.34     7
2156  2020  48085   Dec  47.3      Collin    Texas    3.09     7
2157  2021  48085   Dec  59.7      Collin    Texas    0.93     7
2158  2022  48085   Dec  48.4      Collin    Texas    2.53     7
2159  2023  48085   Dec  50.1      Collin    Texas    3.41     7

[2160 rows x 8 columns]


In [177]:
import pandas as pd

# Assuming top10_migration is your DataFrame containing the necessary columns

# Create a new column for year-month
top10_migration['year_month'] = top10_migration['year'].astype(str) + '-' + top10_migration['month']

# Calculate overall averages for each month across all years
overall_avg = top10_migration.groupby(['month', 'year']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
overall_avg.columns = ['month', 'year', 'overall_avg_temp', 'overall_avg_percip']

# Merge overall averages with the monthly data
monthly_averages = pd.merge(top10_migration, overall_avg, on=['month', 'year'])

# Calculate absolute differences from overall averages
monthly_averages['temp_diff'] = abs(monthly_averages['temp'] - monthly_averages['overall_avg_temp'])
monthly_averages['percip_diff'] = abs(monthly_averages['percip'] - monthly_averages['overall_avg_percip'])

# Weights for desirability components
w_T = 0.3  # weight for temperature
w_P = 0.2  # weight for precipitation
w_r = 0.5  # weight for rank

# Normalize the rank so that a lower rank yields a higher desirability contribution
monthly_averages['rank_adjusted'] = monthly_averages['Rank'].max() - monthly_averages['Rank'] + 1

# Calculate the desirability score for each month
monthly_averages['desirability_month'] = (
    w_T * monthly_averages['temp_diff'] +
    w_P * monthly_averages['percip_diff'] +
    w_r * monthly_averages['rank_adjusted']
)

# Calculate yearly desirability score by taking the mean of monthly scores per county and year
yearly_desirability = monthly_averages.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']

# Merge yearly desirability back into the monthly averages DataFrame
monthly_averages = pd.merge(monthly_averages, yearly_desirability, on=['County_Name', 'State', 'year'], how='left')

# Output the final results
final_results = monthly_averages[['County_Name', 'State', 'temp', 'percip', 'desirability_month', 'Rank', 'month', 'year', 'desirability_year']]
print(final_results)

# Visualization Example (Optional)
import plotly.express as px

# Plot the desirability scores over time
fig = px.line(
    monthly_averages,
    x='year_month',
    y='desirability_month',
    color='County_Name',
    title='Monthly Desirability Scores by County',
    labels={'desirability_month': 'Desirability Score'},
    markers=True
)

# Customize x-axis
fig.update_xaxes(
    tickvals=monthly_averages['year_month'][::6],  # Adjust the frequency as needed
    ticktext=monthly_averages['year_month'][::6]   # Show every 6th month (or adjust as necessary)
)


# Show the figure
fig.show()



      County_Name           State  temp  percip  desirability_month  Rank  \
0        Maricopa         Arizona  53.9    0.52            7.170889     1   
1           Duval         Florida  53.2    1.15            2.834889     9   
2      Miami-Dade         Florida  65.7    2.28            7.094444     8   
3             Ada           Idaho  28.7    1.83            9.151111     4   
4           Clark          Nevada  44.8    0.09            5.613556     2   
...           ...             ...   ...     ...                 ...   ...   
2155        Clark          Nevada  49.6    0.11            6.118444     2   
2156  New Hanover  North Carolina  50.0    8.31            4.194889     6   
2157         Wake  North Carolina  47.1    7.51            2.904889    10   
2158        Bexar           Texas  56.8    0.93            5.821111     3   
2159       Collin           Texas  50.1    3.41            2.808444     7   

     month  year  desirability_year  
0      Jan  2004           7.079852  

In [192]:
import pandas as pd
import plotly.express as px

# Assuming top10_migration is your DataFrame containing the necessary columns

# Create a new column for year-month and convert to datetime
top10_migration['year_month'] = pd.to_datetime(top10_migration['year'].astype(str) + '-' + top10_migration['month'])

# Calculate overall averages for each month across all years
overall_avg = top10_migration.groupby(['month', 'year']).agg({'temp': 'mean', 'percip': 'mean'}).reset_index()
overall_avg.columns = ['month', 'year', 'overall_avg_temp', 'overall_avg_percip']

# Merge overall averages with the monthly data
monthly_averages = pd.merge(top10_migration, overall_avg, on=['month', 'year'])

# Calculate absolute differences from overall averages
monthly_averages['temp_diff'] = abs(monthly_averages['temp'] - monthly_averages['overall_avg_temp'])
monthly_averages['percip_diff'] = abs(monthly_averages['percip'] - monthly_averages['overall_avg_percip'])

# Weights for desirability components
w_T = 0.3  # weight for temperature
w_P = 0.2  # weight for precipitation
w_r = 0.5  # weight for rank

# Normalize the rank so that a lower rank yields a higher desirability contribution
monthly_averages['rank_adjusted'] = monthly_averages['Rank'].max() - monthly_averages['Rank'] + 1

# Calculate the desirability score for each month
monthly_averages['desirability_month'] = (
    w_T * monthly_averages['temp_diff'] +
    w_P * monthly_averages['percip_diff'] +
    w_r * monthly_averages['rank_adjusted']
)

# Calculate yearly desirability score by taking the mean of monthly scores per county and year
yearly_desirability = monthly_averages.groupby(['County_Name', 'State', 'year']).agg({'desirability_month': 'mean'}).reset_index()
yearly_desirability.columns = ['County_Name', 'State', 'year', 'desirability_year']

# Merge yearly desirability back into the monthly averages DataFrame
monthly_averages = pd.merge(monthly_averages, yearly_desirability, on=['County_Name', 'State', 'year'], how='left')

# Sort the data by 'year_month'
monthly_averages.sort_values('year_month', inplace=True)

# Output the final results
final_results = monthly_averages[['County_Name', 'State', 'temp', 'percip', 'desirability_month', 'Rank', 'month', 'year', 'desirability_year']]
print(final_results)




      County_Name           State  temp  percip  desirability_month  Rank  \
0        Maricopa         Arizona  53.9    0.52            7.170889     1   
7           Bexar           Texas  54.2    2.41            6.170444     3   
6            Wake  North Carolina  38.0    1.31            3.409556    10   
5     New Hanover  North Carolina  42.1    2.39            4.243111     6   
8          Collin           Texas  46.4    3.24            2.623111     7   
...           ...             ...   ...     ...                 ...   ...   
2153   Miami-Dade         Florida  70.1    3.98            6.805556     8   
2152        Duval         Florida  58.9    6.26            3.401556     9   
2158        Bexar           Texas  56.8    0.93            5.821111     3   
2154          Ada           Idaho  35.2    1.81            9.098444     4   
2159       Collin           Texas  50.1    3.41            2.808444     7   

     month  year  desirability_year  
0      Jan  2004           7.079852  

/var/folders/q2/p4k6j1ss1qd27364rjxv2fxh0000gn/T/ipykernel_1859/88770279.py:7: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
